###  importing relevant libraries

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from collections import Counter
from tabulate import tabulate
from random import random
from tqdm import tqdm
import seaborn as sns
import pandas as pd
import numpy as np
%matplotlib inline
import warnings
import sklearn
import time
import os
warnings.filterwarnings("ignore")
from sklearn.metrics import confusion_matrix
#time.sleep(3600)

### Discovering Labels

In [3]:
def target_name(name):
    df = pd.read_csv(name,usecols=["Label"])
    df["Label"]=df["Label"].replace(renamer)
    df=df.dropna()
    target_names=sorted(list(df["Label"].unique()))
    return target_names

In [4]:
def folder(f_name): #this function creates a folder.
    try:
        if not os.path.exists(f_name):
            os.makedirs(f_name)
    except OSError:
        print ("The folder could not be created!")

In [5]:
def find_the_way(path,file_format,con=""):
    files_add = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if file_format in file:
                if con in file:
                    files_add.append(os.path.join(r, file))  
            
    return files_add

In [6]:
find_the_way("small",".csv")

['small\\uk-vpn.csv', 'small\\uk.csv', 'small\\us-vpn.csv', 'small\\us.csv']

In [7]:
renamer={}

In [8]:
altime=0
#def most_frequent(List): 
#    return max(set(List), key = List.count) 


 
def most_frequent(List):
    occurence_count = Counter(List)
    occurence_count={k: v for k, v in sorted(occurence_count.items(), key=lambda item: item[1],reverse=True)}
    big=list(occurence_count.values())
    big=big.count(big[0])
    return list(occurence_count.keys())[np.random.randint(big)]


def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(n))
def create_exception(df): 
    exception_list=[]
    dominant_mac=[]
    for i in df['aggregated'].unique():
        k=df[df['aggregated']==i]
        for ii in ['eth.src']:
            hist = {}
            for x in k[ii].values:
                hist[x] = hist.get(x, 0) + 1
            hist=dict(sorted(hist.items(), key=lambda item: item[1],reverse=True))
            temp=next(iter(hist))
            if temp not in dominant_mac:
                dominant_mac.append(temp)
            else:
                exception_list.append(temp)
    return exception_list




def merged(m_test,predict,step,mixed):
    second=time.time()
    mac_test=[]
    for q in m_test.index:
        mac_test.append(m_test[q])

    d_list=sorted(list(m_test.unique()))
    devices={}
    for q in d_list:
        devices[q]=[]    


    new_y=[0]*len(m_test)

    for q,qq in enumerate (mac_test):
        devices[qq].append(q)
    for q in devices:
        a = [devices[q][j:j + step] for j in range(0, len(devices[q]), step)]  
        for qq in a:
            step_list=[]
            for qqq in qq:
                step_list.append(predict[qqq])
            add=most_frequent(list(step_list))
            for qqq in qq:
                new_y[qqq]=add
    results=pd.DataFrame(m_test)
    results["aggregated"]=new_y
    results["normal"]=predict
    
    #MIXED METHOD
    if mixed:
        exception=create_exception(results)
        for q in exception:
            results.loc[results.MAC == q, 'aggregated'] = results['normal']

    return results["aggregated"].values,time.time()-second

## Calculation of evaluations

In [9]:
def score(altime,train_time,test_time,predict,y_test,class_based_results,i,cv,dname,ml):

    rc=sklearn.metrics.recall_score(y_test, predict,average= "macro")
    pr=sklearn.metrics.precision_score(y_test, predict,average= "macro")
    f_1=sklearn.metrics.f1_score(y_test, predict,average= "macro")     
    accuracy=sklearn.metrics.accuracy_score(y_test, predict)
    accuracy_b=sklearn.metrics.balanced_accuracy_score( y_test,predict)
    kappa=sklearn.metrics.cohen_kappa_score(y_test, predict,labels=None, weights=None, sample_weight=None)
    try:
        roc=sklearn.metrics.roc_auc_score(y_test, predict)
    except:roc=0
    report = sklearn.metrics.classification_report(y_test, predict, target_names=target_names,output_dict=True)
    cr = pd.DataFrame(report).transpose()
    line=[dname,i,cv,ml,accuracy,accuracy_b,pr,rc,f_1,kappa,roc,train_time,test_time,altime]

    if class_based_results.empty:
        class_based_results =cr
    else:
        class_based_results = class_based_results.add(cr, fill_value=0)
    return line,class_based_results

In [10]:


def Diff(li1, li2):
    li_dif = [i for i in li1 + li2 if i not in li1 or i not in li2]
    return li_dif
 
# Driver Code
#li1 = [10, 15, 20, 25, 30, 35, 40]
#li2 = [25, 40, 35]


In [11]:
def MLCV(loop1,loop2,output_csv1,cols,step,mixed,dname):

    
    for ii in ml_list:

        class_based_results=pd.DataFrame()#"" #pd.DataFrame(0, index=np.arange((len(target_names)+3)), columns=["f1-score","precision","recall","support"])
        cm=pd.DataFrame()
        cv=0
        output_csv=output_csv1.replace("ML",ii)
        fold=3
        repetition=1
        lines=[["Dataset","T","CV","ML","Acc","b_Acc","Prec","Rec","F1","kap","ROC","tra-T","test-T","Al-Time"]]
        for i in range(repetition):


            rnd = random()
            
            kfold = sklearn.model_selection.KFold(n_splits=fold, shuffle=True, random_state=int(rnd*100))  
            cv=0
            df = pd.read_csv(loop1,usecols=cols)#,header=None )
            ##df = df.reset_index(drop=True)
            df=df.fillna(0)
            macs=df["eth.src"]
            del df["eth.src"]
            #del df["MAC"] # if dataset has MAC colomn please uncomment this line
            X =df[df.columns[0:-1]]
            X=np.array(X)
            df[df.columns[-1]] = df[df.columns[-1]].astype('category')
            y=df[df.columns[-1]].cat.codes  
            X.shape

            for train_index, test_index in kfold.split(X):
                #try:
                X_train, X_test = X[train_index], X[test_index]
                y_train, y_test = y[train_index], y[test_index] 
                m_train, m_test = macs[train_index], macs[test_index] 

            
            

                results_y=[]
                cv+=1
                results_y.append(y_test)
    
    
                #machine learning algorithm is applied in this section
                clf = ml_list[ii]#choose algorithm from ml_list dictionary
                second=time.time()
                clf.fit(X_train, y_train)
                train_time=(float((time.time()-second)) )
                second=time.time()
                predict =clf.predict(X_test)
                test_time=(float((time.time()-second)) )
                if step==1:
                    altime=0
                    line,class_based_results=score(altime,train_time,test_time,predict,y_test,class_based_results,i,cv,dname,ii)
                else:
                    predict,altime=merged(m_test,predict,step,mixed)
                    line,class_based_results=score(altime,train_time,test_time,predict,y_test,class_based_results,i,cv,dname,ii)
                    
                    
                lines.append(line)
                df_cm = pd.DataFrame(confusion_matrix(y_test, predict))
                if cm.empty:
                    cm =df_cm
                else:
                    cm = cm.add(df_cm, fill_value=0)
                
        class_based_results=class_based_results/(repetition*fold)
        results = pd.DataFrame (lines[1:], columns = lines[0])
        results.to_csv(output_csv.replace("ML",ii),index=False)
        results=results.round(3)
        print (tabulate(results, headers=list(results.columns)))
        print()

        
        print (tabulate(class_based_results, headers=list(class_based_results.columns)))
        class_based_results.to_csv(output_csv.replace(".csv","CB.csv"))
        if True:
            cm=cm//(repetition*fold)
            cm.to_csv(output_csv.replace(".csv","CM.csv"))
            #graph_name=output_csv+"_confusion matrix.pdf"   
            #plt.figure(figsize = (40,28))
            #sns.heatmap(cm,xticklabels=target_names, yticklabels=target_names, annot=True, fmt='g')
            #plt.savefig(graph_name,bbox_inches='tight')#, dpi=400)
            #plt.show()
            #print(cm)
            #print("\n\n\n")             


            #plt.clf()

            
            
            
            
            



In [12]:
def ML(loop1,loop2,output_csv1,cols,step,mixed,dname):

    
    for ii in ml_list:

        class_based_results=pd.DataFrame()#"" #pd.DataFrame(0, index=np.arange((len(target_names)+3)), columns=["f1-score","precision","recall","support"])
        cm=pd.DataFrame()
        cv=0
        output_csv=output_csv1.replace("ML",ii)
        repetition=5
        lines=[["Dataset","T","CV","ML","Acc","b_Acc","Prec","Rec","F1","kap","ROC","tra-T","test-T","Al-Time"]]
        for i in range(repetition):



            #TRAIN
            df = pd.read_csv(loop1,usecols=cols)
            df=df.dropna()

            m_train=df["eth.src"]
            del df["eth.src"]
            
            df["Label"]=df["Label"].replace(renamer)
            X_train =df[df.columns[0:-1]]
            X_train=np.array(X_train)
            df[df.columns[-1]] = df[df.columns[-1]].astype('category')
            y_train=df[df.columns[-1]].cat.codes  
            
            

            #TEST
            df = pd.read_csv(loop2,usecols=cols)
            df=df.dropna()

            m_test=df["eth.src"]
            del df["eth.src"]
            
            df["Label"]=df["Label"].replace(renamer)
            #df = shuffle(df, random_state=42)

            X_test =df[df.columns[0:-1]]
            X_test=np.array(X_test)
            df[df.columns[-1]] = df[df.columns[-1]].astype('category')
            y_test=df[df.columns[-1]].cat.codes


            results_y=[]
            cv+=1
            results_y.append(y_test)


            #machine learning algorithm is applied in this section
            clf = ml_list[ii]#choose algorithm from ml_list dictionary
            second=time.time()
            clf.fit(X_train, y_train)
            train_time=(float((time.time()-second)) )
            second=time.time()
            predict =clf.predict(X_test)
            test_time=(float((time.time()-second)) )
            if step==1:
                altime=0
                line,class_based_results=score(altime,train_time,test_time,predict,y_test,class_based_results,i,cv,dname,ii)
            else:
                predict,altime=merged(m_test,predict,step,mixed)
                line,class_based_results=score(altime,train_time,test_time,predict,y_test,class_based_results,i,cv,dname,ii)
                
                
            lines.append(line)
            df_cm = pd.DataFrame(confusion_matrix(y_test, predict))
            if cm.empty:
                cm =df_cm
            else:
                cm = cm.add(df_cm, fill_value=0)
            
        class_based_results=class_based_results/(repetition)
        results = pd.DataFrame (lines[1:], columns = lines[0])
        results.to_csv(output_csv.replace("ML",ii),index=False)
        results=results.round(3)
        print (tabulate(results, headers=list(results.columns)))
        print()
  
        
        print (tabulate(class_based_results, headers=list(class_based_results.columns)))
        class_based_results.to_csv(output_csv.replace(".csv","CB.csv"))
        if True:
            cm=cm//repetition
            cm.to_csv(output_csv.replace(".csv","CM.csv"))
            #graph_name=output_csv+"_confusion matrix.pdf"   
            #plt.figure(figsize = (40,28))
            #sns.heatmap(cm,xticklabels=target_names, yticklabels=target_names, annot=True, fmt='g')
            #plt.savefig(graph_name,bbox_inches='tight')#, dpi=400)
            #plt.show()
            #print(cm)
            #print("\n\n\n")             


            #plt.clf()

            
            
            
            
            



# Machine learning applications 

# Aalto Dataset

In [13]:
feature =  ['HH_0.01_covariance_0_1',
  'HH_0.01_mean_0',
  'HH_0.01_radius_0_1',
  'HH_0.01_std_0',
  'HH_0.1_covariance_0_1',
  'HH_0.1_magnitude_0_1',
  'HH_0.1_std_0',
  'HH_0.1_weight_0',
  'HH_1_magnitude_0_1',
  'HH_1_pcc_0_1',
  'HH_1_radius_0_1',
  'HH_1_std_0',
  'HH_1_weight_0',
  'HH_3_covariance_0_1',
  'HH_3_magnitude_0_1',
  'HH_3_radius_0_1',
  'HH_3_std_0',
  'HH_3_weight_0',
  'HH_5_mean_0',
  'HH_5_pcc_0_1',
  'HH_5_std_0',
  'HH_5_weight_0',
  'HH_jit_0.01_std',
  'HH_jit_0.1_mean',
  'HH_jit_0.1_std',
  'HH_jit_1_mean',
  'HH_jit_1_std',
  'HH_jit_1_weight',
  'HH_jit_3_std',
  'HH_jit_5_std',
  'HpHp_0.01_magnitude_0_1',
  'HpHp_0.01_radius_0_1',
  'HpHp_0.01_weight_0',
  'HpHp_0.1_covariance_0_1',
  'HpHp_0.1_magnitude_0_1',
  'HpHp_0.1_mean_0',
  'HpHp_0.1_pcc_0_1',
  'HpHp_0.1_std_0',
  'HpHp_0.1_weight_0',
  'HpHp_1_covariance_0_1',
  'HpHp_1_magnitude_0_1',
  'HpHp_1_mean_0',
  'HpHp_1_pcc_0_1',
  'HpHp_1_radius_0_1',
  'HpHp_1_std_0',
  'HpHp_3_pcc_0_1',
  'HpHp_3_radius_0_1',
  'HpHp_5_pcc_0_1',
  'HpHp_5_radius_0_1',
  'HpHp_5_std_0',
  'HpHp_5_weight_0',
  'MI_dir_0.01_mean',
  'MI_dir_0.01_std',
  'MI_dir_0.01_weight',
  'MI_dir_0.1_mean',
  'MI_dir_0.1_std',
  'MI_dir_0.1_weight',
  'MI_dir_1_std',
  'MI_dir_1_weight',
  'MI_dir_3_mean',
  'MI_dir_5_mean',
  'MI_dir_5_std',
  'MI_dir_5_weight',"eth.src",
  'Label']



In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
import xgboost

# Cross-Validation

In [15]:

ml_list={#"DT":DecisionTreeClassifier()}
    "RF":RandomForestClassifier(bootstrap=False,criterion="gini",max_depth=27,max_features=9,min_samples_split=8,n_estimators=143)}


In [16]:
file_list={"UKVPN":'small/uk-vpn.csv',
           "UK":'small/uk.csv',
           "USVPN":'small/us-vpn.csv',
           "US":'small/us.csv'}

### Normal

In [17]:
savehere="results/cv/1/"
folder(savehere)
step=1
mixed=0
for file in file_list:
    print(file)
    train=file_list[file]
    test=file_list[file]
    output_csv=f"{savehere}{file}_ML_.csv"
    target_names=target_name(file_list[file])
    MLCV(train,test,output_csv,feature,step,mixed,file)   


UKVPN
    Dataset      T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  ---------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  UKVPN        0     1  RF    0.987    0.916   0.941  0.916  0.926  0.982      0  805.865     2.288          0
 1  UKVPN        0     2  RF    0.987    0.919   0.949  0.919  0.932  0.982      0  785.413     2.106          0
 2  UKVPN        0     3  RF    0.986    0.902   0.935  0.902  0.915  0.981      0  798.009     2.22           0

                      precision    recall    f1-score       support
------------------  -----------  --------  ----------  ------------
appletv                0.973999  0.963969    0.968911    459.667
blink-camera           0.990889  0.994412    0.992645   3101.67
blink-security-hub     0.915428  0.909261    0.912189    139.333
echodot                0.857537  0.823462    0.840102    804.333
echoplus               0.800911  

### Aggregated

In [18]:
savehere="results/cv/13/"
folder(savehere)
step=13
mixed=0
for file in file_list:
    print(file)
    train=file_list[file]
    test=file_list[file]
    output_csv=f"{savehere}{file}_ML_.csv"
    target_names=target_name(file_list[file])
    MLCV(train,test,output_csv,feature,step,mixed,file)   


UKVPN
    Dataset      T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  ---------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  UKVPN        0     1  RF        1    1       1      1      1          1      0  866.068     2.249      1.391
 1  UKVPN        0     2  RF        1    0.991   1      0.991  0.995      1      0  847.98      2.359      1.557
 2  UKVPN        0     3  RF        1    0.992   0.999  0.992  0.995      1      0  841.128     2.289      1.398

                      precision    recall    f1-score       support
------------------  -----------  --------  ----------  ------------
appletv                1         1           1           459.667
blink-camera           1         1           1          3101.67
blink-security-hub     1         0.995402    0.997685    139.333
echodot                1         1           1           804.333
echoplus               0.999454  

# Session vs Session

In [19]:
file_list={"UKVPN@UK":['small/uk-vpn.csv','small/uk.csv'],
           "UK@UKVPN":['small/uk.csv','small/uk-vpn.csv'],
  "USVPN@US":['small/us-vpn.csv','small/us.csv'],
           "US@USVPN":['small/us.csv','small/us-vpn.csv']}

In [20]:
### Normal

In [21]:
say=0
savehere="results/ss/1/"
folder(savehere)


for ii in file_list:
        print()
        say+=1
        train=file_list[ii][0]
        test=file_list[ii][1]
        mixed=False
        step=1
        sayac=1
        output_csv=f"{savehere}/ML@{ii}.csv"
        print(ii)
    
        target_names=target_name(test)
        ML(train,test,output_csv,feature,step,mixed,ii)   


UKVPN@UK
    Dataset      T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  ---------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  UKVPN@UK     0     1  RF    0.882    0.729   0.734  0.729  0.707  0.845      0  1307.22     7.519          0
 1  UKVPN@UK     1     2  RF    0.876    0.731   0.736  0.731  0.708  0.838      0  1304.28     7.99           0
 2  UKVPN@UK     2     3  RF    0.884    0.73    0.735  0.73   0.708  0.847      0  1300.55     7.57           0
 3  UKVPN@UK     3     4  RF    0.875    0.737   0.739  0.737  0.713  0.835      0  1347.24     7.676          0
 4  UKVPN@UK     4     5  RF    0.878    0.735   0.738  0.735  0.712  0.839      0  1356.38     7.256          0

                      precision     recall    f1-score        support
------------------  -----------  ---------  ----------  -------------
appletv                0.768599  0.546748    0.638858     

### Aggregated

In [22]:
say=0
savehere="results/ss/13/"
folder(savehere)


for ii in file_list:
        print()
        say+=1
        train=file_list[ii][0]
        test=file_list[ii][1]
        mixed=False
        step=13
        sayac=1
        output_csv=f"{savehere}/ML@{ii}.csv"
        print(ii)
    
        target_names=target_name(test)
        ML(train,test,output_csv,feature,step,mixed,ii)   


UKVPN@UK
    Dataset      T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  ---------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  UKVPN@UK     0     1  RF    0.946    0.892   0.888  0.892  0.867  0.928      0  1330.28     7.593      3.696
 1  UKVPN@UK     1     2  RF    0.946    0.894   0.891  0.894  0.869  0.929      0  1337.3      7.697      3.699
 2  UKVPN@UK     2     3  RF    0.946    0.893   0.893  0.893  0.87   0.929      0  1348.39     7.424      4.062
 3  UKVPN@UK     3     4  RF    0.946    0.892   0.891  0.892  0.868  0.928      0  1349.11     7.734      3.624
 4  UKVPN@UK     4     5  RF    0.947    0.895   0.891  0.895  0.871  0.929      0  1332.81     7.388      3.701

                      precision    recall    f1-score        support
------------------  -----------  --------  ----------  -------------
appletv                0.964122  0.862702    0.910366    179

# Dataset Vs Dataset

In [23]:
file_list={"UKVPN@USVPN":['small/uk-vpn.csv','small/us-vpn.csv'],
"UKVPN@US":['small/uk-vpn.csv','small/us.csv'],
"UK@USVPN":['small/uk.csv','small/us-vpn.csv'],
"UK@US":['small/uk.csv','small/us.csv'],
"USVPN@UKVPN":['small/us-vpn.csv','small/uk-vpn.csv'],
"USVPN@UK":['small/us-vpn.csv','small/uk.csv'],
"US@UKVPN":['small/us.csv','small/uk-vpn.csv'],
"US@UK":['small/us.csv','small/uk.csv']}

### Normal

In [24]:
say=0
savehere="results/dd/1/"
folder(savehere)
for ii in file_list:
        print()
        say+=1
        train=file_list[ii][0]
        test=file_list[ii][1]
        mixed=False
        step=1
        sayac=1
        output_csv=f"{savehere}/ML@{ii}.csv"
        print(ii)
    
        target_names=target_name(test)
        ML(train,test,output_csv,feature,step,mixed,ii)   


UKVPN@USVPN
    Dataset        T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  UKVPN@USVPN    0     1  RF    0.738    0.495   0.498  0.495  0.443  0.678      0  1267.15     8.737          0
 1  UKVPN@USVPN    1     2  RF    0.73     0.495   0.502  0.495  0.442  0.668      0  1272.31     7.245          0
 2  UKVPN@USVPN    2     3  RF    0.75     0.499   0.503  0.499  0.447  0.692      0  1269.07     7.105          0
 3  UKVPN@USVPN    3     4  RF    0.753    0.501   0.5    0.501  0.448  0.695      0  1263.9      7.08           0
 4  UKVPN@USVPN    4     5  RF    0.711    0.496   0.492  0.496  0.441  0.647      0  1268.08     7.181          0

                      precision      recall    f1-score        support
------------------  -----------  ----------  ----------  -------------
appletv               0.373933   0.5341

### Aggregated

In [ ]:
say=0
savehere="results/dd/13/"
folder(savehere)
for ii in file_list:
        print()
        say+=1
        train=file_list[ii][0]
        test=file_list[ii][1]
        mixed=False
        step=13
        sayac=1
        output_csv=f"{savehere}/ML@{ii}.csv"
        print(ii)
    
        target_names=target_name(test)
        ML(train,test,output_csv,feature,step,mixed,ii)   


UKVPN@USVPN
    Dataset        T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  UKVPN@USVPN    0     1  RF    0.9      0.681   0.706  0.681  0.65   0.873      0  1279.13     7.087      3.317
 1  UKVPN@USVPN    1     2  RF    0.89     0.666   0.698  0.666  0.634  0.862      0  1276.76     7.174      3.477
 2  UKVPN@USVPN    2     3  RF    0.856    0.658   0.689  0.658  0.628  0.82       0  1285.49     7.252      3.477
 3  UKVPN@USVPN    3     4  RF    0.892    0.663   0.698  0.663  0.637  0.864      0  1285.26     7.167      3.875
 4  UKVPN@USVPN    4     5  RF    0.896    0.666   0.697  0.666  0.634  0.869      0  1289.13     9.032      4.625

                      precision    recall    f1-score        support
------------------  -----------  --------  ----------  -------------
appletv                0.691042  0.910163  

In [16]:
file_list={#"UKVPN@USVPN":['small/uk-vpn.csv','small/us-vpn.csv'],
#"UKVPN@US":['small/uk-vpn.csv','small/us.csv'],
#"UK@USVPN":['small/uk.csv','small/us-vpn.csv'],
#"UK@US":['small/uk.csv','small/us.csv'],
#"USVPN@UKVPN":['small/us-vpn.csv','small/uk-vpn.csv'],
"USVPN@UK":['small/us-vpn.csv','small/uk.csv'],
"US@UKVPN":['small/us.csv','small/uk-vpn.csv'],
"US@UK":['small/us.csv','small/uk.csv']}

In [17]:
say=0
savehere="results/dd/13/"
folder(savehere)
for ii in file_list:
        print()
        say+=1
        train=file_list[ii][0]
        test=file_list[ii][1]
        mixed=False
        step=13
        sayac=1
        output_csv=f"{savehere}/ML@{ii}.csv"
        print(ii)
    
        target_names=target_name(test)
        ML(train,test,output_csv,feature,step,mixed,ii)   


USVPN@UK
    Dataset      T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  ---------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  USVPN@UK     0     1  RF    0.899    0.666   0.702  0.666  0.613  0.865      0  1231.65     7.016      3.548
 1  USVPN@UK     1     2  RF    0.899    0.677   0.73   0.677  0.616  0.866      0  1284.91     8.838      3.551
 2  USVPN@UK     2     3  RF    0.9      0.678   0.697  0.678  0.625  0.867      0  1260.87     6.885      3.357
 3  USVPN@UK     3     4  RF    0.899    0.678   0.701  0.678  0.622  0.866      0  1257.03     7.422      3.631
 4  USVPN@UK     4     5  RF    0.898    0.66    0.698  0.66   0.603  0.865      0  1207.86     6.84       3.178

                      precision       recall     f1-score        support
------------------  -----------  -----------  -----------  -------------
appletv                0.364837  1            0.5346

In [18]:
!shutdown /s /t 360